# Chapter 3: Merging Data

[View this lesson on datacamp](https://learn.datacamp.com/courses/merging-dataframes-with-pandas)

In [1]:
import pandas as pd

## pd.merge()
`pd.merge(df1, df2)` is another function that allows you to combine two (or more) DataFrames. 

Recall from the last chapter how we combined favourite colour and birth month data from ten people:

In [2]:
fav_colour = pd.read_csv('fav_colour.csv')
birthday_month = pd.read_csv('birthday_months.csv')

pd.concat([fav_colour, birthday_month], axis=1)

,Participant num,Fav Colour,Participant num,Birthday Month
0,1,blue,1,may
1,2,red,2,june
2,3,green,3,january
3,4,purple,4,february
4,5,red,5,september
5,6,green,6,july
6,7,orange,7,may
7,8,yellow,8,may
8,9,yellow,9,august
9,10,pink,10,december


Doing this with `pd.merge()` is a bit simpler, and has the advantage that it automatically recognizes the fact that both data files have a column called `participant_num()`, and merges these into a single column:

In [3]:
pd.merge(fav_colour, birthday_month)

,Participant num,Fav Colour,Birthday Month
0,1,blue,may
1,2,red,june
2,3,green,january
3,4,purple,february
4,5,red,september
5,6,green,july
6,7,orange,may
7,8,yellow,may
8,9,yellow,august
9,10,pink,december


This works smoothly when we have a common column in both DataFrames, with the same values in that column (in this case, `Participant num` with values 1–10). But what happens when we have non-overlapping data? Here we load eye colour data, which we have for only some of the people that we have favourite colours and birth months for, as well as for some new people:

In [4]:
eye_colour = pd.read_csv('eye_colour.csv')
eye_colour

,Participant num,eye_colour
0,1,brown
1,2,blue
2,3,blue
3,4,hazel
4,5,green
5,11,brown
6,12,brown
7,13,blue


When we merge `eye_colour` with one of the other DataFrames, we only get the data that overlaps between the two (based on shared `Participant num`s):

In [5]:
pd.merge(fav_colour, eye_colour)

,Participant num,Fav Colour,eye_colour
0,1,blue,brown
1,2,red,blue
2,3,green,blue
3,4,purple,hazel
4,5,red,green


In other words, `pd.merge()` uses an **inner join** by default. We can override this using the `how=` argument though:

In [6]:
pd.merge(fav_colour, eye_colour, how='outer')

,Participant num,Fav Colour,eye_colour
0,1,blue,brown
1,2,red,blue
2,3,green,blue
3,4,purple,hazel
4,5,red,green
5,6,green,NaN
6,7,orange,NaN
7,8,yellow,NaN
8,9,yellow,NaN
9,10,pink,NaN


## Merging 'on'

This works fine as long as we are merging two DataFrames that share a column label, *and* have shared values in that label column (e.g., participant numbers 1–5 in the above example). But our data aren't always structured that way. For example, let's re-load the RT data used in the previous chapter, that came from the same participant in two different testing sessions:

In [7]:
sess_1 = pd.read_csv('session_1.csv', index_col='trial')
sess_2 = pd.read_csv('session_2.csv', index_col='trial')
pd.merge(sess_1, sess_2)

,rt


Merging these generates no output (other than the `rt` label). Why not? Let's look at the inputs:

In [8]:
print(sess_1)
print(sess_2)

          rt
trial       
0      0.988
1      0.753
2      0.949
3      0.824
4      0.262
5      0.803
6      0.376
7      0.496
8      0.235
9      0.336
10     0.645
          rt
trial       
0      0.718
1      0.851
2      0.747
3      0.520
4      0.991
5      0.004
6      0.547
7      0.883
8      0.841
9      0.195
10     0.828


Both inputs have a `trial` column, with the same values (0–10). However, they both also have an `rt` column, and the RT values are different for every trial. Since pandas sees the `rt` column label in both columns, it will only do the inner join on rows that match on *both* `trial` *and* `rt`. 

We can override this default behaviour by explicitly telling pandas what columns to merge on; in this case, `trial`. 

In [9]:
sess_12 = pd.merge(sess_1, sess_2, on='trial')
sess_12

,rt_x,rt_y
trial,,
0,0.988,0.718
1,0.753,0.851
2,0.949,0.747
3,0.824,0.520
4,0.262,0.991
5,0.803,0.004
6,0.376,0.547
7,0.496,0.883
8,0.235,0.841


Note that in this case, the identically-named `rt` columns are given distinct names so that we know where they came from (`x` being the first input, and `y` the second). We can replace these with meaningful labels if we like, using the `suffixes=` argument and a list of labels:

In [10]:
sess_12 = pd.merge(sess_1, sess_2, on='trial', suffixes=['_sess_1', '_sess_2'])
sess_12

,rt_sess_1,rt_sess_2
trial,,
0,0.988,0.718
1,0.753,0.851
2,0.949,0.747
3,0.824,0.520
4,0.262,0.991
5,0.803,0.004
6,0.376,0.547
7,0.496,0.883
8,0.235,0.841


`pd.merge` can also come to the rescue if you have matching data columns in two inputs, but the column names aren't the same. It's not uncommon that a researcher will make little errors like capitalizing a title one time, but not another. This happened with the third session from our RT experiment:

In [11]:
sess_3 = pd.read_csv('session_3.csv')
sess_3

,Trial,RT
0,0,0.844168
1,1,0.913048
2,2,0.843295
3,3,0.530306
4,4,0.266715
5,5,0.707006
6,6,0.973193
7,7,0.432562
8,8,0.522106
9,9,0.876626


So when we try to merge this third session with the already-merged other two, we get an error:

In [12]:
pd.merge(sess_12, sess_3)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

So we need to tell pandas which columns in each input to merge on, with `left_on` referring to the left (first) input, and `right_on` referring to the right (second) input:

In [ ]:
pd.merge(sess_12, sess_3, left_on='trial', right_on='Trial')

Note that this matches data between the two inputs, but keeps both columns.

---
## Joining DataFrames

Yet another way of combining pandas DataFrames is with the `.join()` method. While `pd.merge()` is a function (you can tell because the command name, `.merge`, is preceded by `pd` rather than a DataFrame, and all the input data is inside the parentheses), `.join()` is a method and so must be appended to the name of an existing DataFrame, with the DataFrame you want to join to it specified in the parentheses:

In [ ]:
sess_12.join(sess_3)

Note that `.join()` is less picky than `pd.merge`: it ran fine even though there are no exactly-matching column labels shared by the inputs. Indeed, we can join DataFrames that have totally different columns and even lengths:

In [ ]:
sess_12.join(fav_colour)

In other words, `.join()` simply adds columns of the 'right' DataFrme (the one in parentheses) to the columns of the 'left' DataFrame (preceding the dot), lining up the rows and adding extra rows of `NaN`s if the inputs are not the same length. 

This is more flexible, but potentially more messy or dangerous. We want to be certain that the order of inputs in the two DataFrames we're merging is exactly the same to avoid mis-aligning the data. Interestingly, if pandas does note identical column labels in the two DataFrames, it will throw an error because it doesn't know if you want to use those to match rows between the inputs:

In [ ]:
fav_colour = pd.read_csv('fav_colour.csv')
eye_colour = pd.read_csv('eye_colour.csv')

fav_colour.join(eye_colour)

**Indexing** can help `.join()` operate more safely and reliably. If we specify the shared columns as indexes of each DataFrame, pandas will match the inputs based on the indexes:

In [ ]:
fav_colour = fav_colour.set_index('Participant num')
eye_colour = eye_colour.set_index('Participant num')

fav_colour.join(eye_colour)

By default `.join()` uses an outer join, but again we can use an argument to change that behaviour. However, for `.join()` the argument is `how=`:

In [ ]:
fav_colour.join(eye_colour, how='inner')

### Left and Right joins

In addition to `outer` (union; i.e., all inputs) and `inner` (intersection; i.e., only shared input) joins, we can use `left` and `right` arguments to specify including only the indices in one input that match those in the other input. 

So if we use `how=left`, pandas will include all indices present int he left input, filling any non-matches in the right input with `NaN`: 

In [ ]:
fav_colour.join(eye_colour, how='left')

Conversely, with `how=right` we get all indices present in the right input, again filling anything missing from the left with `NaN`. 

In [ ]:
fav_colour.join(eye_colour, how='right')

---
## Which function or method to use?

### How many input DataFrames?
**2 inputs**
- methods: `.append()`; `.join()`
- function: `pd.merge()`

**\>2 inputs:**
- function: `pd.concat()`- because this takes lists as input

### For stacking vertically
(rows on top of rows)

`.append()` 
- for 2 inputs

`.concat()` 
- for >2 inputs
- allows simple inner and outer joins based on indexes

### For stacking horizontally

`.concat()` 
- for >2 inputs
- allows simple inner and outer joins based on indexes

`.join()` 
- allows left and right, as well as inner and outer, joins

`pd.merge()` 
- allows joins on multiple, possibly non-matching, columns